In [1]:
import xml.etree.cElementTree as ET
import urllib2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Enable IPython to display matplotlib graphs

In [2]:
%matplotlib inline

In [3]:
result = pd.DataFrame(columns=('name', 'length', 'date', 'location', 'players'))

In [4]:
api_link = 'https://boardgamegeek.com/xmlapi2/plays'
user = 'banzayats'
mindate='2016-01-01'
maxdate='2016-12-31'
page = 1
end = False

In [5]:
while not end:
    url = '{0}?username={1}&mindate={2}&maxdate={3}&page={4}'.format(api_link, user, mindate, maxdate, page)
    tree = ET.ElementTree(file=urllib2.urlopen(url))
    plays = tree.getroot()
    if len(plays) < 100:
        end = True
    for i,play in enumerate(plays.findall(".//play[@quantity='1']")):
        length = play.attrib['length']
        date = play.attrib['date']
        location = play.attrib['location']
        name = play[0].attrib['name'].encode("utf-8")
        players = len(play.find('players').findall('player'))
        pointer = i + 100 * (page - 1)
        result.loc[pointer] = [name, length, date, location, players]
    page += 1

In [6]:
result.head()

,name,length,date,location,players
0,Castles of Mad King Ludwig,50,2016-12-18,Home,2.0
1,Eldritch Horror,160,2016-12-17,Home,3.0
2,7 Wonders Duel,40,2016-12-16,Home,2.0
3,7 Wonders Duel,40,2016-12-16,Home,2.0
4,7 Wonders Duel,35,2016-12-14,Home,2.0


In [7]:
result.shape

(335, 5)

In [8]:
result['length'] = result['length'].astype(int)
result['date'] = pd.to_datetime(result['date'])
result.dtypes

name                object
length               int64
date        datetime64[ns]
location            object
players            float64
dtype: object

In [9]:
plays = result[['name','length','date']]
games_group = plays.groupby('name')
games_group.size()

name
7 Wonders                                                                   6
7 Wonders Duel                                                              9
Agricola:  All Creatures Big and Small                                      8
Arkham Horror                                                               6
BANG!                                                                       2
BattleLore (Second Edition)                                                23
Between Two Cities                                                          2
Bruges                                                                     22
Brügge: Die Haustiere                                                       1
CV                                                                          6
Castles of Mad King Ludwig                                                  3
Catan                                                                       6
Citadels                                                   

In [10]:
games_length = games_group.sum()
games_length.sort_values('length', ascending=False).head()

,length
name,
BattleLore (Second Edition),2050
Eldritch Horror,1660
Bruges,1615
Runewars,1600
Race for the Galaxy,1390
